# Neural Network to recognize Simpsons characters

In [ ]:
#Imports
import pandas as pd
from pathlib import Path
import os.path
from os.path import isfile, join
from os import listdir
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image
import matplotlib.image as mpimg
import numpy as np
import shutil
import random
from distutils.dir_util import copy_tree
import warnings
from PIL import Image
import os, sys

warnings.filterwarnings('ignore')

In [ ]:
# I will use the Simpsons dataset from kaggle https://www.kaggle.com/alexattia/the-simpsons-characters-dataset
# The concept is that I downloaded the dataset into the folder that this notebook is executing and then from this original folder,I create with code train,validation and test directories with respective files inside it.I decided to train the entire dataset and validate it also and use about 1600 images available for test.
#I will try to implement the notebook over all the simpsons characters, just to identify unknown areas in deep learning and learn more from it.

In [ ]:
#Function to list all files in directory and sub directories
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:                                                                                        
                r.append(os.path.join(subdir, file))                                                                         
    return r    

In [ ]:
# The path to the directory where the original dataset was placed
original_dataset_dir = './dataset/simpsons_dataset/'

# Our working directory
base_dir = './dataset/simpsons/'
if (not os.path.exists(base_dir) ):
    os.mkdir(base_dir)

In [ ]:
#Setup train,validation and test directories
train_dir = os.path.join(base_dir, 'train/')
if (not os.path.exists(train_dir) ):
    os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation/')
if (not os.path.exists(validation_dir) ):
    os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test/')
if (not os.path.exists(test_dir) ):
    os.mkdir(test_dir)


In [ ]:
# Directories for our training, validation and test splits.Copy images inside them

# Get image names(path) to array
simpsons_files = list_files(original_dataset_dir)

#Shuffle array
random.shuffle(simpsons_files)

#Collection to store copied files, so as each folder to have different ones
file_list = []

#Copy entire dataset to train one
copy_tree(original_dataset_dir,train_dir)

#Copy entire dataset to validation one
copy_tree(original_dataset_dir,validation_dir)

#Get some files for test
counter=0
for fname in simpsons_files:
    if fname in file_list:
        continue
    counter+=1
    if (counter==7000):
        break
    dst = os.path.join(test_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [ ]:
print('total validation simpsons classes:', len(os.listdir(train_dir)))
print('total validation simpsons classes:', len(os.listdir(validation_dir)))
print('total test simpsons images:', len(os.listdir(test_dir)))

In [ ]:
train_dataset = './dataset/simpsons_dataset'
test_dataset = './dataset/simpsons_test_dataset'

In [ ]:
#Function to get random image for testing
img_width, img_height = 256, 384

def getRandomImage(path, img_width, img_height):
    """function loads a random images from a random folder in our validation path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width, img_height)), final_path, path_class

In [ ]:
#Select some random images
files = []

for i in range(0, 3):
    path = train_dir#'./dataset/simpsons_dataset/' 
    img, final_path, true_label = getRandomImage(path, img_width, img_height)
    files.append(final_path)
    img=mpimg.imread((files[i]))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
#Image size for regenerator
img_rows, img_cols = 150, 150
batch_size = 20

#Set up train and validation generators
train_data_dir = train_dir#'./dataset/simpsons_dataset'
validation_data_dir = validation_dir#'./dataset/simpsons_validation'

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.3, height_shift_range=0.3, horizontal_flip=True, fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size,class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')

In [ ]:
#Get classes names and number of them
from glob import glob
classes = [i[15:-1].upper() for i in sorted(glob('./dataset//simpsons//train//*//'))]
print("Classes: ",classes)
num_classes = len(classes)
print("Number of classes: ",num_classes)

In [ ]:
#Prepare our model - New one working

from keras.layers import BatchNormalization

model = Sequential()
# First CONV-ReLU Layer
model.add(Conv2D(64, (3, 3), padding = 'same', activation='relu', input_shape = (img_rows, img_cols, 3)))
model.add(BatchNormalization())

# Second CONV-ReLU Layer
model.add(Conv2D(64, (3, 3), padding = "same", activation='relu', input_shape = (img_rows, img_cols, 3)))
model.add(BatchNormalization())

# Max Pooling with Dropout 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# 3rd set of CONV-ReLU Layers
model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())

# 4th Set of CONV-ReLU Layers
model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())

# Max Pooling with Dropout 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# 5th Set of CONV-ReLU Layers
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())

# 6th Set of CONV-ReLU Layers
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())

# Max Pooling with Dropout 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# First set of FC or Dense Layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Second set of FC or Dense Layers
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(47, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
#Create classes for storing our model and its layers,so as to be reusable enough
class SimpsonsModel:
    def __init__(self,train_dir,validation_dir,layers,rows,columns):
        self.train_dir = train_dir
        self.validation_dir = validation_dir
        self.layers = list(layers)
        self.rows = rows
        self.columns = columns

class Layer:
    def __init__(self,layer_type,layer_no,filters,kernel_size,padding,activation,input_shape,max_pooling,pool_size,batch_normalization,dropout,dropout_rate,units):
        self.layer_type = layer_type
        self.layer_no = layer_no
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.padding = padding
        self.activation = activation
        self.input_shape = input_shape
        self.max_pooling = max_pooling
        self.pool_size = pool_size
        self.batch_normalization = batch_normalization
        self.dropout = dropout
        self.dropout_rate = dropout_rate
        self.units = units

In [ ]:
#Function to prepare our model, can be decorated with future layers and custom implementation
def PrepareModel(input_model):
    model = Sequential()

    for layer in input_model.layers:
        print(str(layer.layer_no) + " " + layer.layer_type)
        if (layer.layer_type == "Conv2D"):
            if (layer.input_shape):
                model.add(Conv2D(layer.filters, layer.kernel_size, padding = layer.padding, activation=layer.activation, input_shape = layer.input_shape))
            else:
                model.add(Conv2D(layer.filters, layer.kernel_size, padding = layer.padding, activation=layer.activation))
        if (layer.layer_type == "Flatten"):
            model.add(Flatten())
            continue
        if (layer.layer_type == "Dense"):
            model.add(Dense(layer.units, activation=layer.activation))
        if (layer.batch_normalization):
            model.add(BatchNormalization())
        if (layer.max_pooling):
            model.add(MaxPooling2D(pool_size=layer.pool_size))
        if (layer.dropout):
            model.add(Dropout(layer.dropout_rate))
    return model


In [ ]:
#Instatiate our simpsons model and pass layer parameters to it
#Constructor needs layer_type,layer_no,filters,kernel_size,padding,activation,input_shape,max_pooling,pool_size,batch_normalization,dropout,dropout_rate,units

layers = []

layer1 = Layer("Conv2D",1,64,(3, 3),'same','relu',(img_rows, img_cols, 3),False,(0,0),True,False,0,0)
layer2 = Layer("Conv2D",2,64,(3, 3),'same','relu',(img_rows, img_cols, 3),True,(2, 2),True,True,0.2,0)
layer3 = Layer("Conv2D",3,128,(3, 3),'same','relu',None,False,(0,0),True,False,0,0)
layer4 = Layer("Conv2D",4,128,(3, 3),'same','relu',None,True,(2, 2),True,True,0.2,0)
layer5 = Layer("Conv2D",5,256,(3, 3),'same','relu',None,False,(0,0),True,False,0,0)
layer6 = Layer("Conv2D",6,256,(3, 3),'same','relu',None,True,(2, 2),True,True,0.2,0)
flatten = Layer("Flatten",1,0,(0, 0),'','',None,True,(2, 2),False,False,0.0,0)
dense1 = Layer("Dense",1,0,(0, 0),'','relu',None,False,(0, 0),True,True,0.5,256)
dense2 = Layer("Dense",2,0,(0, 0),'','relu',None,False,(0, 0),True,True,0.5,256)
dense3 = Layer("Dense",3,0,(0, 0),'','sigmoid',None,False,(0, 0),False,False,0.0,num_classes)

layers = [layer1,layer2,layer3,layer4,layer5,layer6,flatten,dense1,dense2,dense3]
simpsonsModel = SimpsonsModel(train_dir,validation_dir,layers,img_rows,img_cols)
model_new = PrepareModel(simpsonsModel)

In [ ]:
#Check that are new model will be the same with the initiated one.Just verify that our function and class instatiation works as expected.
model_new.summary()

In [ ]:
model = model_new

In [ ]:
#Reduce LR , compile model and store callbacks so as to pass them to fit_generator
#Set up train and validation generators
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD, Adam
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_delta=0.00001)
callbacks = [reduce_lr]

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])

# Rescale images by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size,class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')


In [ ]:
# Train our model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=callbacks)

In [ ]:
# Save model
model.save('simpsons_model.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
#val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
#plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# This is module with image preprocessing utilities
from keras.preprocessing import image


datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

fnames = [os.path.join(train_data_dir, fname) for fname in os.listdir(train_data_dir)]
files_1 = list_files(fnames[10])
# We pick one image to "augment"
img_path = files_1[5]#fnames[3]

# Read the image and resize it
img = image.load_img(img_path, target_size=(150, 150))

# Convert it to a Numpy array with shape (150, 150, 3)
x = image.img_to_array(img)

# Reshape it to (1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

# The .flow() command below generates batches of randomly transformed images.
# It will loop indefinitely, so we need to `break` the loop at some point!
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

In [ ]:
#Function to plot training and validation accuracy and loss
def plot_sets(history):
    acc = history.history['accuracy']
    #val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'rs')
    #plt.plot(epochs, val_acc, 'g^')
    plt.title('Training and validation accuracy')

    plt.figure()
    plt.plot(epochs, loss, 'rs')
    plt.plot(epochs, val_loss, 'g^')
    plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')

In [ ]:
plot_sets(history)

In [ ]:
#TASOS 
# We need to recreate our validation generator with shuffle = false
validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(img_rows, img_cols),
                                                              batch_size=batch_size, class_mode='categorical', shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

nb_train_samples = 20933
nb_validation_samples = 20933

#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def show_predictions(predictions,generator):
    labels = [label_map[i] for i in range(len(label_map))]
    y_true = [label_map[i] for i in generator.classes]
    y_pred = [label_map[i] for i in predictions]
    
    print(len(generator.classes))
    print(len(y_pred))
    report = classification_report(y_true, y_pred, labels=labels)
    print (str(report))

    cm = confusion_matrix(y_true, y_pred, labels)
    
    #print(cm)
    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    #plt.title('Confusion matrix of the classifier')
    fig.colorbar(cax)
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, labels, rotation=90)
    plt.yticks(tick_marks, labels)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.show()

In [ ]:
from keras.callbacks import ModelCheckpoint

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=20, 
                                                    shuffle = False)

checkpoint_filepath = "./simpsons_model.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit_generator(train_generator, 
                                  steps_per_epoch=train_generator.n // train_generator.batch_size, 
                                  validation_data=validation_generator, 
                                  validation_steps=validation_generator.n // validation_generator.batch_size, 
                                  epochs=20, 
                                  workers=100, 
                                  shuffle=True,
                                  callbacks=callbacks_list)

# Plot training and dev: loss and accuracy
plot_training_and_dev(history)

# Predict test data
predictions = model.predict_generator(test_generator, steps = test_generator.n // test_generator.batch_size)
predictions = np.argmax(predictions, axis=-1) #multiple categories     
show_predictions(predictions)

In [ ]:
#predictions = model.predict_generator(validation_generator, steps = validation_generator.n // validation_generator.batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size, class_mode='categorical')

predictions = model.predict_generator(validation_generator, steps = 1047)
predictions = np.argmax(predictions, axis=-1) #multiple categories     
label_map = validation_generator.class_indices
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
num_classes = len(label_map)


In [ ]:
show_predictions(predictions,validation_generator)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# Testing Images

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
target_names = list(class_labels.values())

In [ ]:
len(target_names)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

Y_pred = model.predict_generator(validation_generator, 100)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
def TestImage(path,test_model):
    img=mpimg.imread(path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x.astype('float32')/255
    pred1 = np.argmax(test_model.predict(x))
    print("Predicted character:{}.".format(class_labels[pred1])) 

In [ ]:
#Load model
from keras.models import load_model
model1 = load_model('./simpsons_model.h5')


In [ ]:
test_dir

In [ ]:
# Perform test for random images and get predicted results

path = test_dir
for i in range(0,10):
    random_filename = random.choice([
        x for x in os.listdir(test_dir)
        if os.path.isfile(os.path.join(path, x))
    ])
    random_filename = os.path.join(test_dir,random_filename)
    print("Selected filename: " + random_filename)

    TestImage(random_filename,model1)
    print("")

# Use pre-trained model


In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 20933)
validation_features, validation_labels = extract_features(validation_dir, 20933)
test_features, test_labels = extract_features(test_dir, 1600)

In [ ]:
train_features = np.reshape(train_features, (20933, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (20933, 4 * 4 * 512))
#test_features = np.reshape(test_features, (1600, 4 * 4 * 512))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
#model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
#model.add(layers.Dropout(0.5))
#model.add(layers.Dense(1, activation='sigmoid'))

model.add(Dense(256, activation='relu',input_dim=32 * 32 * 512))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu',input_dim=32 * 32 * 512))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

#model.compile(optimizer=optimizers.RMSprop(lr=2e-5),loss='categorical_crossentropy',metrics=['acc'])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])

history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Load the pretained model
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(47, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
model.save("./simpsons2.h5")

In [ ]:
pd.DataFrame(hist.history)[['accuracy']].plot()
plt.title("Accuracy")
plt.show()

In [ ]:
pd.DataFrame(hist.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

# 4. Visualize the result<a class="anchor" id="4"></a>

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f'The first 5 predictions: {pred[:5]}')

In [ ]:
from sklearn.metrics import accuracy_score
y_test = list(test_df.Label)
acc = accuracy_score(y_test,pred)
print(f'Accuracy on the test set: {acc * 100:.2f}%')

In [ ]:
from sklearn.metrics import classification_report
class_report = classification_report(y_test, pred, zero_division=1)
print(class_report)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cf_matrix = confusion_matrix(y_test, pred, normalize='true')
plt.figure(figsize = (20,15))
sns.heatmap(cf_matrix, annot=False, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)))
plt.title('Normalized Confusion Matrix', fontsize = 23)
plt.show()

# Use of pre-trained model

In [ ]:
from keras.applications import MobileNet

conv_base = MobileNet(weights='imagenet',include_top=False,input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 1024))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 20933)
validation_features, validation_labels = extract_features(validation_dir, 20933)
#test_features, test_labels = extract_features(test_dir, 1000)

In [ ]:
train_dir.count

In [ ]:
train_features = np.reshape(train_features, (20933, 4 * 4 * 1024))
validation_features = np.reshape(validation_features, (20933, 4 * 4 * 1024))
#test_features = np.reshape(test_features, (1000, 4 * 4 * 1024))

# 5. Examples of prediction<a class="anchor" id="5"></a>


In [ ]:
# Expose the real character values and the prediction ones
counter=0

for filename in test_df.Filepath:
    counter+=1
    print("Real Character: " + test_df.Label.iloc[counter] + " Predicted: " + pred[counter])

In [ ]:
# Display 15 picture of the dataset with their labels
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 12),subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[i]))
    ax.set_title(f"True: {test_df.Label.iloc[i].split('_')[0]}\nPredicted: {pred[i].split('_')[0]}")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.image as mpimg
from keras.preprocessing import image

def post_validation(path):
    img=mpimg.imread(path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
    # prediction
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x.astype('float32')/255
    pred1 = np.argmax(model.predict(x))
    pred2 = np.argmax(model.predict(x))

In [ ]:
post_validation('./dataset/check/Abraham_Simpson.png')

In [ ]:
post_validation('./dataset/check/unnamed.jpg')